In [ ]:
import pandas as pd
import numpy as np
from pygam import GAM, s
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import TheilSenRegressor
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('Q_Donnees.csv')

# Préparation des données
variables_meteo = ['RR', 'TM', 'FFM', 'DXY']
variable_pollution = 'CNO2'
df = df.dropna(subset=variables_meteo + [variable_pollution])
df['AAAAMMJJ'] = pd.to_datetime(df['AAAAMMJJ'], format='%Y-%m-%d', errors='coerce')
# Conversion des dates

df.set_index('AAAAMMJJ', inplace=True)
date_test = df.index.values

print(date_test)
df.head()


In [ ]:

df_train_val = df[(df.index.year >= 1995) & (df.index.year <= 2016)]

# Filtrage des données pour les années 2016 à 2020 pour le test
df_test = df[(df.index.year >= 2020) & (df.index.year <= 2023)]


X_train_val = df_train_val[variables_meteo].values
y_train_val = df_train_val[variable_pollution].values
X_test = df_test[variables_meteo].values
y_test = df_test[variable_pollution].values


In [ ]:
gam = GAM(s(0) + s(1) + s(2) + s(3))
gam.fit(X_train_val, y_train_val)

In [ ]:
# Prédiction sur les données d'entraînement et de test
y_train_val_pred = gam.predict(X_train_val)
y_test_pred = gam.predict(X_test)

# Calcul des résidus
effects_train_val = y_train_val - y_train_val_pred
effects_test = y_test - y_test_pred




In [ ]:
print(date_test)

In [ ]:
# Déterminer la pente (slope) et l'ordonnée à l'origine pour Theil-Sen
slope = -0.020335539886242354  # Exemple de pente de Theil-Sen
intercept = np.median(effects_test) - slope * np.median(np.arange(len(effects_test)))

# Générer la tendance estimée avec Theil-Sen
trend_line = slope * np.arange(len(date_test)) + intercept

date_test = date_test[-len(effects_test:]

len(trend_line)

plt.figure(figsize=(10, 5))
plt.plot(date_test, effects_test, label='Effets dus aux actions humaines', color='red')
plt.xlabel('Date')
plt.ylabel('Effets des actions humaines sur la concentration en NO2')
plt.title('Effets des actions humaines sur la Prédiction de la Concentration en NO2 avec GAM')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Déterminer la pente (slope) et l'ordonnée à l'origine pour Theil-Sen
slope = -0.020335539886242354  # Exemple de pente de Theil-Sen
intercept = np.median(effects_test) - slope * np.median(np.arange(len(effects_test)))

# Générer la tendance estimée avec Theil-Sen
trend_line = slope * np.arange(len(date_test)) + intercept
plt.xlabel('Date')
plt.ylabel('Effets des actions humaines sur la concentration en NO2')
plt.title('Effets des actions humaines sur la Prédiction de la Concentration en NO2 avec GAM')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Vérification de l'absence de tendance
if np.abs(slope) < 1e-3:
    print("Les effets des actions humaines n'ont pas de tendance significative, indiquant une bonne décorrélation.")
else:
    print("Les effets des actions humaines ont une tendance significative, indiquant une possible mauvaise décorrélation.")

In [ ]:
# Affichage des résultats
plt.figure(figsize=(10, 5))
plt.plot(date_test, effects_test, label='Effets dus aux actions humaines', color='red')
plt.plot(date_test, trend_line[:len(date_test)], label='Tendance estimée avec Theil-Sen', color='blue', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Effets des actions humaines sur la concentration en NO2')
plt.title('Effets des actions humaines sur la Prédiction de la Concentration en NO2 avec GAM')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Vérification de l'absence de tendance
if np.abs(slope) < 1e-3:
    print("Les effets des actions humaines n'ont pas de tendance significative, indiquant une bonne décorrélation.")
else:
    print("Les effets des actions humaines ont une tendance significative, indiquant une possible mauvaise décorrélation.")